# Optional Assignment 2
A general expression for the local truncation error for any $r$-step linear multistep method is: 
$$
\begin{align}
\tau(t_{n+r}) & = \frac 1k \bigg[\sum_{j=0}^r \alpha_j u(t_{n+j}) + k\sum_{j=0}^r \beta_j u'(t_{n+j})\bigg] \\ 
& = \frac1k\bigg[\sum_{j=0}^r \alpha_{j} \bigg] + \frac1k\bigg[\sum_{j=0}^r (j\alpha_{j} - \beta_{j}) \bigg] + \cdots + k^{q-1} \bigg[\sum_{j=0}^r \Big(\frac{1}{q!} j^q \alpha_j - \frac{1}{(q-1)!} j^{q-1}\beta_j \Big)\bigg] u^{(q)} (t_n) + \cdots
\end{align}
$$ 

<!-- Let's name each coefficient $\{C_r\}_{r=0}^\infty = \sum_{j=0}^r \Big(\frac{1}{q!} j^q \alpha_j - \frac{1}{(q-1)!} j^{q-1}\beta_j \Big)$ and $C(\alpha_j) = \frac{1}{q!} j^q \alpha_j$  and $C(\beta_j) = \frac{1}{(q-1)!} j^{q-1}\beta_j$ -->

To achieve the maximum efficiency possible, we want to banish the coefficients <!-- $C_0, \ldots, C_r$ --> up to order $r$. We can translate this problem into this system of linear equations
$$\begin{bmatrix}
1 & 0 & 1 & 0 & \cdots & 1 & 0 \\
0 & -1 & 1 & -1 & \cdots & r & -1 \\
0 & 0 & \frac12 & -1 & \cdots & \frac12 r^2 & -r \\
\vdots &\vdots&\vdots&\vdots&\ddots&\vdots&\vdots \\
0 & 0 & \frac{1}{r!} & -\frac{1}{(r-1)!} & \cdots & \frac{1}{r!} r^{r} & -\frac{1}{(r-1)!} r^{r-1}  \\
\end{bmatrix}
\begin{bmatrix}
\alpha_0 \\ \beta_0 \\ \alpha_1 \\ \beta_1 \\ \vdots \\ \alpha_r \\ \beta_r 
\end{bmatrix}
= 
\begin{bmatrix}
0 \\ 0 \\ 0 \\ 0 \\ \vdots \\ 0 \\ 0
\end{bmatrix}
$$
which has infinite solutions at the moment.

## Adams-Bashforth
For Adams-Bashforth method, we choose $\alpha_r = 1$, $\alpha_{r-1} = -1$, $\alpha_j = 0$ for $j<r-1$ and $\beta_{r} = 0$, then we define a system of linear equations such that 
$$\begin{bmatrix}
0 & 0 & \cdots & 1 & 0 \\
-1 & -1 & \cdots & -1 & -1 \\
0 & -1 & \cdots & -(r-2) & -(r-1) \\
\vdots&\vdots&\ddots&\vdots&\vdots \\
0 & -\frac{1}{(r-1)!} & \cdots & -\frac{1}{(r-1)!} (r-2)^{r-1} & -\frac{1}{(r-1)!} (r-1)^{r-1}  \\
\end{bmatrix}
\begin{bmatrix}
\beta_0 \\ \beta_1 \\ \vdots \\ \beta_{r-2}  \\ \beta_{r-1} 
\end{bmatrix}
= 
\begin{bmatrix}
0 \\ -1 \\ \frac12(r-1)^2-\frac12r^2\\ \vdots \\ \frac{1}{r!}(r-1)^{r}-\frac{1}{r!}r^{r} 
\end{bmatrix}
$$

In [1]:
# Import Libraries
import numpy as np
from scipy.special import factorial

def c_alpha(q: int, j: int) -> float:
    return j**q/factorial(q) if q!=0 else 1

def c_beta(q: int, j: int) -> float:
    return -(j)**(q-1)/factorial(q-1) if q!=0 else 0

In [ ]:
def ab_eq(order: int, q: int) -> list[float]:
    if order < 1:
        raise ValueError("r must be positive")

    Ai = [0] * order
    for j in range(order):
        Ai[j] = c_beta(q, j)
 
    bi = c_alpha(q, order-1) - c_alpha(q, order)
    return Ai, bi

def adams_bashforth(order: int):
    if order < 1:
        raise ValueError("r must be positive")

    A, b = [], []
    for i in range(1, order+1):
        Ai, bi = ab_eq(order, i)
        A.append(Ai)
        b.append(bi)

    A = np.array(A)
    b = np.array(b)
    x = np.linalg.solve(A, b)

    return x

The coefficients of Adams-Bashforth up to order 5 are:

In [3]:
print(f'ADAMS-BASHFORTH COEFFICIENTS')
for i in range(1, 5):
    print(f'r={i}: {adams_bashforth(i)}')

ADAMS-BASHFORTH COEFFICIENTS
r=1: (array([[-1.]]), array([-1.]), array([1.]))
r=2: (array([[-1., -1.],
       [ 0., -1.]]), array([-1. , -1.5]), array([-0.5,  1.5]))
r=3: (array([[-1. , -1. , -1. ],
       [ 0. , -1. , -2. ],
       [ 0. , -0.5, -2. ]]), array([-1.        , -2.5       , -3.16666667]), array([ 0.41666667, -1.33333333,  1.91666667]))
r=4: (array([[-1.        , -1.        , -1.        , -1.        ],
       [ 0.        , -1.        , -2.        , -3.        ],
       [ 0.        , -0.5       , -2.        , -4.5       ],
       [ 0.        , -0.16666667, -1.33333333, -4.5       ]]), array([-1.        , -3.5       , -6.16666667, -7.29166667]), array([-0.375     ,  1.54166667, -2.45833333,  2.29166667]))


We can verify that the coefficients are indeed correct.

## Adams Moulton
For Adams-Bashforth method, we choose $\alpha_r = 1$, $\alpha_{r-1} = -1$ and $\alpha_j = 0$ for $j<r-1$. We need to integrate one more coefficient into the SLE.

In [ ]:
def am_eq(order: int, q: int) -> list[float]:
    if order < 1:
        raise ValueError("r must be positive")

    Ai = [0] * (order+1)
    for j in range(order+1):
        Ai[j] = c_beta(q, j)
 
    bi = c_alpha(q, order-1) - c_alpha(q, order)
    return Ai, bi

def adams_moulton(order: int):
    if order < 1:
        raise ValueError("r must be positive")

    A, b = [], []
    for i in range(1, order+2):
        Ai, bi = am_eq(order, i)
        A.append(Ai)
        b.append(bi)

    A = np.array(A)
    b = np.array(b)
    x = np.linalg.solve(A, b)

    return x 

In [11]:
print(f'ADAMS-MOULTON COEFFICIENTS')
for i in range(1, 5):
    print(f'r={i}: {adams_moulton(i)}')

ADAMS-MOULTON COEFFICIENTS
r=1: [0.5 0.5]
r=2: [-0.08333333  0.66666667  0.41666667]
r=3: [ 0.04166667 -0.20833333  0.79166667  0.375     ]
r=4: [-0.02638889  0.14722222 -0.36666667  0.89722222  0.34861111]


## Backward Differentation
For Backward Differentation method, we choose $\beta_j = 0$ for $j<r$ so that 
$$\sum_{j=0}^r \alpha_j U_{n+j} = k\beta_r f(U_{n+r}, t_{n+r})$$
therefore

the output is $$[\alpha_0, \ldots, \alpha_r, \beta_r]$$

In [ ]:
def bd_eq(order: int, q: int) -> list[float]:
    if order < 1:
        raise ValueError("Order must be positive")

    Ai = [0] * (order+1)
    for j in range(order+1):
        Ai[j] = c_alpha(q, j)

    return Ai, -c_beta(q, order)

def backward_differentation(order: int):
    if order < 1:
        raise ValueError("Order must be positive")

    A, b = [], []
    for i in range(order+1):
        Ai, bi = bd_eq(order, i)
        A.append(Ai)
        b.append(bi)

    A = np.array(A)
    b = np.array(b)
    x = np.linalg.solve(A, b)

    return x

In [32]:
print(f'BACKWARD DIFFERENTIATION COEFFICIENTS')
for i in range(1, 5):
    print(f'r={i}: {backward_differentation(i)}')

BACKWARD DIFFERENTIATION COEFFICIENTS
r=1: [-1.  1.]
r=2: [ 0.5 -2.   1.5]
r=3: [-0.33333333  1.5        -3.          1.83333333]
r=4: [ 0.25       -1.33333333  3.         -4.          2.08333333]
